In [1]:
!pip install pandas requests python-dotenv

In [25]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import logging

# Load API key from environment file
load_dotenv("API_KEY.env")
API_KEY = os.getenv("SIMFIN_API_KEY")

# Base URLs for SimFin API
BASE_URL_PRICES = "https://backend.simfin.com/api/v3/companies/prices/compact"
BASE_URL_STATEMENTS = "https://backend.simfin.com/api/v3/companies/statements/compact"

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class PySimFin:
    def __init__(self):
        """Initialize API headers with authentication."""
        if not API_KEY:
            raise ValueError("API Key not found. Ensure 'API_KEY.env' contains SIMFIN_API_KEY.")
        self.headers = {
            "Authorization": f"api-key {API_KEY}",
            "Accept": "application/json"
        }

    def get_share_prices(self, ticker: str, start: str) -> pd.DataFrame:
        """
        Fetch daily share prices for a given company from the given start date.

        Args:
            ticker (str): Stock ticker symbol (e.g., "AAPL")
            start (str): Start date in "YYYY-MM-DD" format

        Returns:
            pd.DataFrame: Cleaned DataFrame containing stock prices.
        """
        params = {"ticker": ticker, "start": start}
        
        try:
            response = requests.get(BASE_URL_PRICES, headers=self.headers, params=params)
            response.raise_for_status()
            
            data = response.json()
            if isinstance(data, list) and len(data) > 0:
                data = data[0]  # Extract dictionary from list
            
            columns = data["columns"]  # Get column names
            price_data = data["data"]  # Get stock price data
            
            # Convert to DataFrame
            df = pd.DataFrame(price_data, columns=columns)

            # Rename and clean up columns
            df = df.rename(columns={"Date": "date"})
            df["date"] = pd.to_datetime(df["date"])
            df = df.set_index("date")
            df = df.sort_index()

            return df
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching share prices for {ticker}: {e}")
            return pd.DataFrame()

    def get_financial_statement(self, ticker: str, statements: str, period: str) -> pd.DataFrame:
        """
        Fetch financial statements for a given company.

        Args:
            ticker (str): Stock ticker symbol (e.g., "AAPL")
            statements (str): Type of financial statement ("pl", "bs", "cf")
            period (str): Reporting period ("q1", "q2", "q3", "q4", "fy")

        Returns:
            pd.DataFrame: DataFrame containing the financial statement data.
        """
        params = {
            "ticker": ticker,
            "statements": statements,
            "period": period
        }
        
        try:
            response = requests.get(BASE_URL_STATEMENTS, headers=self.headers, params=params)
            response.raise_for_status()
            
            data = response.json()
            if isinstance(data, list) and len(data) > 0:
                data = data[0]  # Extract first item if response is a list

            # Extract the first financial statement (e.g., Profit & Loss)
            if "statements" in data and len(data["statements"]) > 0:
                statement_data = data["statements"][0]
                
                # Extract columns & actual financial data
                columns = statement_data["columns"]
                financial_data = statement_data["data"]
                
                # Convert to DataFrame
                df = pd.DataFrame(financial_data, columns=columns)
                
                # Convert Report Date column if present
                if "Report Date" in df.columns:
                    df["Report Date"] = pd.to_datetime(df["Report Date"])
                    df = df.set_index("Report Date")
                    df = df.sort_index()

                return df
            else:
                logging.error(f"No financial statement data found for {ticker}.")
                return pd.DataFrame()
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching financial statement for {ticker}: {e}")
            return pd.DataFrame()

# Example usage
if __name__ == "__main__":
    api = PySimFin()
    
    # Fetch stock prices for Apple (AAPL) from Jan 1, 2023
    df_prices = api.get_share_prices("AAPL", "2023-01-01")
    print("Share Prices:")
    print(df_prices.head())
    
    # Fetch annual income statement for Apple (AAPL) for fiscal year 2023
    df_income_statement = api.get_financial_statement("AAPL", statements="pl", period="fy")
    print("\nIncome Statement:")
    print(df_income_statement.head())

Share Prices:
            Dividend Paid  Common Shares Outstanding  Last Closing Price  \
date                                                                       
2023-01-03            NaN                15892723000              125.07   
2023-01-04            NaN                15892723000              126.36   
2023-01-05            NaN                15892723000              125.02   
2023-01-06            NaN                15892723000              129.62   
2023-01-09            NaN                15892723000              130.15   

            Adjusted Closing Price  Highest Price  Lowest Price  \
date                                                              
2023-01-03                  123.64         130.90        124.17   
2023-01-04                  124.91         128.66        125.08   
2023-01-05                  123.59         127.77        124.76   
2023-01-06                  128.13         130.29        124.89   
2023-01-09                  128.66         133.41  